# Imports de llibreries

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# Càrrega dels datasets

In [48]:
df_proposals = pd.read_csv("./data/participa.gencat.cat-open-data-proposals.csv")
df_comments = pd.read_csv("./data/participa.gencat.cat-open-data-proposal_comments.csv")

# Exploratory Data Analysis

In [49]:
print(df_proposals.columns)
print(f"Nombre de columnes: {len(df_proposals.columns)}")
print(f"Dimensió del dataset: {df_proposals.shape}")

Index(['id', 'category/id', 'category/name/ca', 'category/name/es',
       'category/name/oc', 'scope/id', 'scope/name/en', 'scope/name/ca',
       'scope/name/es', 'scope/name/oc', 'participatory_space/id',
       'participatory_space/url', 'component/id', 'title/ca', 'body/ca',
       'address', 'latitude', 'longitude', 'state', 'reference', 'answer/en',
       'answer/ca', 'answer/es', 'answer/oc', 'supports',
       'endorsements/total_count', 'endorsements/user_endorsements',
       'comments', 'attachments', 'followers', 'published_at', 'url',
       'meeting_urls', 'related_proposals', 'is_amend',
       'original_proposal/title', 'original_proposal/url', 'category/name/en',
       'title/oc', 'body/oc', 'title/es', 'body/es',
       'original_proposal/title/ca', 'title/en', 'body/en'],
      dtype='object')
Nombre de columnes: 45
Dimensió del dataset: (10440, 45)


Observem que el dataset de propostes té 45 columnes diferents i un total de 10440 files. Per poder seleccionar només les files que ens interessen, agafarem el subset que tingui la url de l'espai participatori corresponent a l'Assemblea Ciutadana pel Clima de Catalunya. En aquest cas és la columna `participatory_space/url`, i la url en concret `http://participa.gencat.cat/processes/assembleaclima?participatory_process_slug=assembleaclima`.

In [50]:
df_proposals = df_proposals[df_proposals['participatory_space/url'] == 'http://participa.gencat.cat/processes/assembleaclima?participatory_process_slug=assembleaclima']
df_proposals.head(5)

,id,category/id,category/name/ca,category/name/es,category/name/oc,scope/id,scope/name/en,scope/name/ca,scope/name/es,scope/name/oc,...,original_proposal/title,original_proposal/url,category/name/en,title/oc,body/oc,title/es,body/es,original_proposal/title/ca,title/en,body/en
9622,87446,2540.0,Desplegament de les energies renovables,Despliegue de las energías renovables,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9623,87447,2540.0,Desplegament de les energies renovables,Despliegue de las energías renovables,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9624,87450,2540.0,Desplegament de les energies renovables,Despliegue de las energías renovables,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9625,87451,2541.0,Model d'alimentació pel futur,Modelo de alimentación para el futuro,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9626,87452,2541.0,Model d'alimentació pel futur,Modelo de alimentación para el futuro,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observem a primera vista que existeixen moltes columnes on els valors són NaNs i que potser no ens interessen gaire. Per tant, doncs, eliminem les columnes que continguin íntegrament valors NaNs

In [51]:
df_proposals = df_proposals.dropna(axis=1)
print(f"Nou nombre de columnes: {len(df_proposals.columns)}")
df_proposals.head(2)

Nou nombre de columnes: 15


,id,participatory_space/id,participatory_space/url,component/id,title/ca,body/ca,reference,supports,endorsements/total_count,comments,attachments,followers,published_at,url,is_amend
9622,87446,562,http://participa.gencat.cat/processes/assemble...,3827,Participar Assemblea Ciutadana pel Clima de Ca...,"Buenos días, me gustaría participar en la asam...",II-PROP-2023-10-87446,0,4,0,0,2,2023-10-09 13:27:07 +0200,http://participa.gencat.cat/processes/assemble...,False
9623,87447,562,http://participa.gencat.cat/processes/assemble...,3827,Assemblea ciutadana pel clima de Catalunya,M'agradaria participar en aquesta assemblea pe...,II-PROP-2023-10-87447,0,1,1,0,3,2023-10-10 10:38:57 +0200,http://participa.gencat.cat/processes/assemble...,False


Un cop eliminades les columnes que només contenien valors NaNs, es redueix el nombre de columnes de 45 a només 15. Una reducció significativa.
Seguim amb el tractament ara del dataset dels comentaris a les propostes.